# Tensorboard setup

This notebook demonstrate the use of Tensorboard to monitor and visualize model training.  We implement a simple Multi-Layer Perceptron model to predict house prices. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

2025-05-02 14:29:00.431771: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-02 14:29:00.443118: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746210540.451685   46986 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746210540.454192   46986 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746210540.460984   46986 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## Data preprocessing

In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

In [3]:
X = housing.data
y = housing.target

In [4]:
from sklearn.model_selection import train_test_split

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)
del X_train_full, y_train_full

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

## Model

In [6]:
num_features = X_train.shape[1]

nn = keras.models.Sequential([
    keras.layers.Dense(50, activation='relu', input_shape = (num_features,)),
    keras.layers.Dense(1)
])

/media/hieuluu/MAIN_DATA/deeplearning/venv-deeplearning/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-02 14:29:02.263946: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-05-02 14:29:02.263961: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-05-02 14:29:02.263963: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: PC-Linux-Mint
2025-05-02 14:2

In [7]:
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 50)             │           450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 501 (1.96 KB)

 Trainable params: 501 (1.96 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
nn.compile(loss="mean_squared_error",
           optimizer="sgd",
           metrics=["mape"])

## Configure TensorBoard

In [9]:
def get_run_logdir():
    import time, os
    run_id = time.strftime("run_%Y%m%d-%H%M%S")
    return os.path.join(os.pardir, "logs", run_id)

run_logdir = get_run_logdir()

In [10]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [11]:
history = nn.fit(X_train, y_train, epochs=30,
                 validation_data = (X_valid, y_valid),
                 callbacks=[tensorboard_cb])

Epoch 1/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 838us/step - loss: 1.2744 - mape: 43.2363 - val_loss: 0.4706 - val_mape: 28.9558
Epoch 2/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - loss: 0.4767 - mape: 29.1426 - val_loss: 1.0601 - val_mape: 29.6014
Epoch 3/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - loss: 0.4495 - mape: 27.2022 - val_loss: 16.9099 - val_mape: 36.2066
Epoch 4/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 646us/step - loss: 0.4264 - mape: 27.1218 - val_loss: 26.5740 - val_mape: 38.9157
Epoch 5/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 662us/step - loss: 0.4305 - mape: 26.7964 - val_loss: 3.4408 - val_mape: 31.4397
Epoch 6/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 639us/step - loss: 0.4143 - mape: 26.7117 - val_loss: 0.4394 - val_mape: 27.1827
Epoch 7/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - loss: 0.4013 - mape: 26.5127 - val_loss: 0.3906 - val_mape: 26.8327
Epoch 8/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 645us/step - loss: 0.3912 - mape: 25.8299 - val_loss: 0.3631 - val_mape: 25.4031
Epoch 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=../logs --port=6006